In [1]:
%pip install llama-index llama-index-llms-openai

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   ----- ---------------------------------- 0.3/2.1 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.1 MB 399.6 kB/s eta 0:00:04
   ---------- ----------------------------- 0.5/2.1 MB 399.6 kB/s eta 0:00:04
   ---------- ----------------------------- 0.5/2.1 MB 399.6 kB/s eta 0:00:04
   ---------- ----------------------------- 0.5/2.1 MB 399.6 kB/s eta 0:00:04
   ---------- ----------------------------- 0.5/2.1 MB 399.6 kB/s eta 0:00:04
   ---------- ----------------------------- 0.5/2.1 MB 399.6 kB/s eta 0:00:04
   ---------- --------------------------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
prefect 2.10.8 requires coolname>=1.0.4, which is not installed.
prefect 2.10.8 requires pathspec>=0.8.0, which is not installed.
prefect 2.10.8 requires python-slugify>=5.0, which is not installed.
prefect 2.10.8 requires readchar>=4.0.0, which is not installed.
embedchain 0.1.103 requires alembic<2.0.0,>=1.13.1, but you have alembic 1.8.1 which is incompatible.
embedchain 0.1.103 requires langchain<0.2.0,>=0.1.4, but you have langchain 0.3.3 which is incompatible.
fastapi-utils 0.2.1 requires pydantic<2.0,>=1.0, but you have pydantic 2.9.2 which is incompatible.
fastapi-utils 0.2.1 requires sqlalchemy<2.0.0,>=1.3.12, but you have sqlalchemy 2.0.36 which is incompatible.
prefect 2.10.8 requires sqlalchemy[asyncio]!=1.4.33,<2.0,>=1.4.22, but you have sqlalchemy 2.0.36 which is incompatible.
sqlmodel 0.0.8 requires

In [3]:
import os
import openai
from sqlalchemy import (
    create_engine,
    MetaData,
    Table, 
    Column,
    String,
    Integer,
    select
)

In [4]:
engine = create_engine("sqlite:///:memory:")

In [5]:
metadata_obj = MetaData()

In [7]:
table_name = "office_stats"
office_stats_table = Table(
    table_name,
    metadata_obj,
    Column("emp_name", String(16), primary_key=True),
    Column("salary", Integer),
    Column("dept", String(16)),
)
metadata_obj.create_all(engine)

In [8]:
from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI

In [28]:
os.environ["OPENAI_API_KEY"] = "sk-.."
openai.api_key = os.environ["OPENAI_API_KEY"]

In [29]:
llm = OpenAI(base_url="http://localhost:1234/v1", api_key="lm-studio")

In [10]:
sql_database = SQLDatabase(
    engine,
    include_tables=["office_stats"]
)

In [11]:
from sqlalchemy import insert

rows = [
    {
        "emp_name":"John",
        "salary":15000,
        "dept": "IT"
    },
    {
        "emp_name":"Jane",
        "salary":20000,
        "dept": "HR"
    },
    {
        "emp_name":"Elliot",
        "salary":10000,
        "dept": "IT"
    },
    {
        "emp_name":"Drake",
        "salary":25000,
        "dept": "OPS"
    },
    {
        "emp_name":"Cooper",
        "salary":30000,
        "dept": "DATA"
    },
    {
        "emp_name":"Hillman",
        "salary":17000,
        "dept": "IT"
    },
]

In [12]:
for row in rows:
    stmt = insert(office_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [18]:
from sqlalchemy import text
with engine.begin() as connection:
    query = text("SELECT * FROM office_stats")
    res = connection.execute(query)
    rows = res.fetchall()
    for row in rows:
        print(row)

('John', 15000, 'IT')
('Jane', 20000, 'HR')
('Elliot', 10000, 'IT')
('Drake', 25000, 'OPS')
('Cooper', 30000, 'DATA')
('Hillman', 17000, 'IT')


In [30]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=["office_stats"], llm=llm
)
query_str = "Which employee has the highest salary ?"
response = query_engine.query(query_str)

Retrying llama_index.llms.openai.base.OpenAI._chat in 0.77691409676824 seconds as it raised AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: lm-studio. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}.
Retrying llama_index.llms.openai.base.OpenAI._chat in 0.7528112881917137 seconds as it raised AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: lm-studio. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}.
Retrying llama_index.llms.openai.base.OpenAI._chat in 2.4837795044477358 seconds as it raised AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: lm-studio. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error',

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: lm-studio. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [27]:
display(Markdown(f"{response}"))

NameError: name 'Markdown' is not defined